In [63]:
import concurrent.futures
import logging
import pandas as pd
import os

In [64]:
from retrieve_data_by_query import RetrieveDataByQuery
# t = RetrieveDataByQuery(sessionid="1653774166%3A9uDdOCkxtjfjYr%3A25%3AAYf2jGK4vIx5W_dkE5m8a7PpoN4LA2D5CxUpR4C7BA")
# t = RetrieveDataByQuery(sessionid="65955050144%3ACYLJ62xy3SlmZY%3A21%3AAYczdKeVZcK3hiLzXoCohMHegWecE3FIvmjZ_Myz8w")
# t = RetrieveDataByQuery(sessionid="71441509544%3A9NroGpYJCtlrjn%3A1%3AAYe0D7Oa0fAH8aWyYbNn37a64dbS-vNqOjYqoxqPtA")
# t = RetrieveDataByQuery(sessionid="71441509544%3AInt8o2FRhY0K8X%3A8%3AAYcj0-033-xr3LXSw-hiJIHbtefXYJ6bmhU5hXgnyw")
# t = RetrieveDataByQuery(sessionid="71441509544%3Agmkrt0BbmNRnLU%3A11%3AAYea6rZE7RRYmc3iLzMfkMOgTLtmAcXwi1-pqwTPy")
# t = RetrieveDataByQuery(sessionid="5955050144%3AgXGX0LNgbSQuMP%3A17%3AAYceXPyYrX6JIg6LdqPxK1kCxMYpDAWnjntpQ155Uw")
t = RetrieveDataByQuery(sessionid="65955050144%3AgXGX0LNgbSQuMP%3A17%3AAYceXPyYrX6JIg6LdqPxK1kCxMYpDAWnjntpQ155Uw")
# print(t.retrieve_follower_by_userid("187264532"))
# t.retrieve_follower_by_userid_after("187264532", str(25 * 1))

In [65]:
query = "Artificial Intelligence"

In [66]:
def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


def get_all_follower(userid: str):
    results = []
    try:
        response = t.retrieve_follower_by_userid(userid)
        data = response.get("data", {}).get("user", {}).get("followers", {})
        results.extend(get_follower_data(data.get("edges", [])))
    
        has_next_page = data.get("page_info", {}).get("has_next_page", False)
        batch_number = 1
    
        while has_next_page:
            try:
                print("Pagina: ", batch_number)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                results.extend(get_follower_data(data.get("edges", [])))
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
            except Exception as e:
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")

    except Exception as e:
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    
    return results


def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

In [67]:


# results = []
# response = t.retrieve_thread_by_query(query)
# data = response.get("data", {}).get("searchResults", {})
# results.extend(get_post_data(data.get("edges", [])))
# has_next_page = data.get("page_info", {}).get("has_next_page", False)
# after = data.get("page_info", {}).get("end_cursor")
# i = 1000

# while has_next_page and i > 0:
#     response = t.retrieve_thread_by_query_after(query, after)
#     data = response.get("data", {}).get("searchResults", {})
#     results.extend(get_post_data(data.get("edges", [])))
#     has_next_page = data.get("page_info", {}).get("has_next_page", False)
#     after = data.get("page_info", {}).get("end_cursor")
#     i -= 1

# # Creazione del DataFrame
# df = pd.DataFrame(results)
# df.shape

In [68]:
# df.drop_duplicates(inplace=True)
# df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')
# df.to_csv("data.csv", index=False)

In [69]:
df = pd.read_csv("data.csv")
df['user_pk'] = df['user_pk'].astype(str)
unique_user_pk = df['user_pk'].unique()
print(len(unique_user_pk))
unique_user_pk = unique_user_pk[unique_user_pk != '46153597924']
len(unique_user_pk)

874


873

In [70]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "followers_data.csv"
processed_users = load_processed_users(csv_file)

In [71]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler())

def fetch_followers(pk):
    logging.info(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"]
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None



follower_list = []
batch_size = 2
i = 0

logging.info("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        logging.info(f"Skipping already processed user_pk: {user_pk}")
        continue
    
    processed_users.update([user_pk])
    results = fetch_followers(user_pk)
    if results is None:
        break
    follower_list.extend(results)
    i += 1
    
    if i >= batch_size:
        logging.info(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []  
        i = 0

if follower_list:
    logging.info(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)
    save_to_csv(follower_df, csv_file)

logging.info("Follower data collection completed")

Pagina:  1
Pagina:  2
Pagina:  3
Pagina:  4
Pagina:  5
Pagina:  6
Pagina:  7
Pagina:  8
Pagina:  9
Pagina:  10
Pagina:  11
Pagina:  12
Pagina:  13
Pagina:  14
Pagina:  15
Pagina:  16
Pagina:  17
Pagina:  18
Pagina:  19
Pagina:  20
Pagina:  21
Pagina:  22
Pagina:  23
Pagina:  24
Pagina:  25
Pagina:  26
Pagina:  27
Pagina:  28
Pagina:  29
Pagina:  30
Pagina:  31
Pagina:  32
Pagina:  33
Pagina:  34
Pagina:  35
Pagina:  36
Pagina:  37
Pagina:  38
Pagina:  39
Pagina:  40
Pagina:  41
Pagina:  42
Pagina:  43
Pagina:  44
Pagina:  45
Pagina:  46
Pagina:  47
Pagina:  48
Pagina:  49
Pagina:  50
Pagina:  51
Pagina:  52
Pagina:  53
Pagina:  54
Pagina:  55
Pagina:  56
Pagina:  57
Pagina:  58
Pagina:  59
Pagina:  60
Pagina:  61
Pagina:  62
Pagina:  63
Pagina:  64
Pagina:  65
Pagina:  66
Pagina:  67
Pagina:  68
Pagina:  69
Pagina:  70
Pagina:  71
Pagina:  72
Pagina:  73
Pagina:  74
Pagina:  75
Pagina:  76
Pagina:  77
Pagina:  78
Pagina:  79
Pagina:  80
Pagina:  81
Pagina:  82
Pagina:  83
Pagina:  84
P

KeyboardInterrupt: 

In [12]:
df1 = pd.read_csv("followers_data.csv")
df_count= df1.groupby("user_pk").count()
print(df_count.shape)
df_count

(17, 3)


,follower_pk,follower_username,follower_count
user_pk,,,
187264532,217,217,217
241752404,49,49,49
273811002,167,167,167
295418393,1530,1530,1530
429303617,264,264,264
623659295,627,627,627
1175659953,247,247,247
1624593034,50,50,50
3312160968,433,433,433
